In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import VQC
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import wandb
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 19:12:15.493751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 19:12:15.493797: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 19:12:20.260832: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 19:12:20.260872: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 19:12:20.261229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [3]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.pca = 4
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-2
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.feature_map = 'DoubleAngleMap'
args.ansatz = 'TTN'

In [4]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Performing PCA on data...
Cumulative sum on train : 55.622551253000125
Cumulative sum on test: 55.622551253000125
Standardizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 2, 2, 1) │ (634, 2, 2, 1) │ (2115, 2, 2, 1) │ (2, 2, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═════

In [5]:
data.config()

{'input_dims': (2, 2, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [6]:
model = VQC(data.config(), args)

In [7]:
model.compile()
model.build_graph().summary()

Model: "VQC-DoubleAngleMap-TTN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 2, 1)]         0         
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 22        
                                                                 
Total params: 22
Trainable params: 14
Non-trainable params: 8
_________________________________________________________________


In [8]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
188/188 [==============================] - 17s 89ms/step - loss: 0.7250 - q_auc: 0.8155 - custom_accuracy: 0.7295 - val_loss: 0.6158 - val_q_auc: 0.8734 - val_custom_accuracy: 0.7795 - lr: 0.0100
Epoch 2/10
188/188 [==============================] - 16s 87ms/step - loss: 0.5654 - q_auc: 0.8852 - custom_accuracy: 0.8007 - val_loss: 0.5080 - val_q_auc: 0.9140 - val_custom_accuracy: 0.8303 - lr: 0.0100
Epoch 3/10
188/188 [==============================] - 16s 86ms/step - loss: 0.5096 - q_auc: 0.9049 - custom_accuracy: 0.8289 - val_loss: 0.4755 - val_q_auc: 0.9228 - val_custom_accuracy: 0.8489 - lr: 0.0100
Epoch 4/10
188/188 [==============================] - 16s 86ms/step - loss: 0.4835 - q_auc: 0.9111 - custom_accuracy: 0.8407 - val_loss: 0.4545 - val_q_auc: 0.9276 - val_custom_accuracy: 0.8523 - lr: 0.0100
Epoch 5/10
188/188 [==============================] - 16s 87ms/step - loss: 0.4665 - q_auc: 0.9143 - custom_accuracy: 0.8478 - val_loss: 0.4436 - val_q_auc: 0.9287 - val_cu

In [9]:
model.test(data, callbacks)

34/34 [==============================] - 2s 64ms/step - loss: 0.4215 - q_auc: 0.9221 - custom_accuracy: 0.8801


[0.421548455953598, 0.9220749735832214, 0.8800551295280457, 0.9220749735832214]

In [10]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.pca = 16
args.normalize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.hinge_labels = 1
args.use_quantum = 1

# Base Model
args.learning_rate = 1e-2
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.feature_map = 'AmplitudeMap'
args.ansatz = 'TTN'

In [11]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Performing PCA on data...
Cumulative sum on train : 77.73337564866492
Cumulative sum on test: 77.73337564866492
Normalizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 4, 4, 1) │ (634, 4, 4, 1) │ (2115, 4, 4, 1) │ (4, 4, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031,)         │ (634,)         │ (2115,)         │ (1,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪═

In [12]:
data.config()

{'input_dims': (4, 4, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [13]:
model = VQC(data.config(), args)
model.compile()
model.build_graph().summary()

../../qml_hep_lhc/encodings/amplitude.py:13: UserWarning: AmplitudeMap currently does not normalize the input unless padding is needed.
User must manually normalize the input.
  "AmplitudeMap currently does not normalize the input unless padding is needed.\nUser must manually normalize the input."


Model: "VQC-AmplitudeMap-TTN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4, 4, 1)]         0         
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 21        
                                                                 
Total params: 21
Trainable params: 6
Non-trainable params: 15
_________________________________________________________________


In [14]:
model.fit(data, callbacks)

Epoch 1/10
188/188 [==============================] - 39s 206ms/step - loss: 0.5728 - q_auc_1: 0.9580 - custom_accuracy: 0.8572 - val_loss: 0.5482 - val_q_auc_1: 0.9648 - val_custom_accuracy: 0.8612 - lr: 0.0100
Epoch 2/10
188/188 [==============================] - 39s 206ms/step - loss: 0.5394 - q_auc_1: 0.9736 - custom_accuracy: 0.8729 - val_loss: 0.5474 - val_q_auc_1: 0.9649 - val_custom_accuracy: 0.8590 - lr: 0.0100
Epoch 3/10
188/188 [==============================] - 38s 205ms/step - loss: 0.5390 - q_auc_1: 0.9739 - custom_accuracy: 0.8733 - val_loss: 0.5460 - val_q_auc_1: 0.9663 - val_custom_accuracy: 0.8659 - lr: 0.0100
Epoch 4/10
188/188 [==============================] - 39s 206ms/step - loss: 0.5391 - q_auc_1: 0.9741 - custom_accuracy: 0.8734 - val_loss: 0.5483 - val_q_auc_1: 0.9639 - val_custom_accuracy: 0.8675 - lr: 0.0100
Epoch 5/10
188/188 [==============================] - 37s 198ms/step - loss: 0.5391 - q_auc_1: 0.9739 - custom_accuracy: 0.8729 - val_loss: 0.5487 - val

In [15]:
model.test(data, callbacks)

34/34 [==============================] - 5s 133ms/step - loss: 0.5325 - q_auc_1: 0.9689 - custom_accuracy: 0.8735


[0.5324534177780151,
 0.9688942432403564,
 0.8734681010246277,
 0.9688942432403564]